# 📊 ML Trading Strategy - Interactive Tutorial

Este notebook demonstra o uso completo do framework de estratégias de trading com Machine Learning.

## Índice
1. [Carregamento de Dados](#1-carregamento-de-dados)
2. [Análise Exploratória](#2-análise-exploratória)
3. [Engenharia de Features](#3-engenharia-de-features)
4. [Treinamento de Modelos](#4-treinamento-de-modelos)
5. [Backtesting](#5-backtesting)
6. [Análise de Resultados](#6-análise-de-resultados)

In [ ]:
# Imports
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("✅ Bibliotecas importadas com sucesso!")

## 1. Carregamento de Dados

Vamos começar carregando dados históricos de preços usando o Yahoo Finance.

In [ ]:
from utils.data_loader import DataLoader

# Inicializar o loader
loader = DataLoader()

# Carregar dados
ticker = "AAPL"  # Altere para o ticker desejado
data = loader.download_stock_data(ticker, period="5y")

print(f"Dados carregados para {ticker}")
print(f"Período: {data['date'].min()} até {data['date'].max()}")
print(f"Total de dias: {len(data)}")

# Visualizar primeiras linhas
data.head()

## 2. Análise Exploratória

Vamos explorar os dados e visualizar tendências.

In [ ]:
# Estatísticas descritivas
print("📊 Estatísticas Descritivas:")
print(data[['open', 'high', 'low', 'close', 'volume']].describe())

In [ ]:
# Visualizar preço de fechamento
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Preço de fechamento
axes[0].plot(data['date'], data['close'], label='Close Price', linewidth=1.5)
axes[0].set_title(f'{ticker} - Preço de Fechamento', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Data')
axes[0].set_ylabel('Preço ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Volume
axes[1].bar(data['date'], data['volume'], alpha=0.5, label='Volume')
axes[1].set_title(f'{ticker} - Volume de Negociação', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Data')
axes[1].set_ylabel('Volume')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calcular retornos
data['returns'] = data['close'].pct_change()

# Visualizar distribuição de retornos
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
axes[0].hist(data['returns'].dropna(), bins=50, alpha=0.7, edgecolor='black')
axes[0].set_title('Distribuição de Retornos Diários', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Retorno')
axes[0].set_ylabel('Frequência')
axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(data['returns'].dropna())
axes[1].set_title('Box Plot dos Retornos', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Retorno')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Retorno médio diário: {data['returns'].mean():.4f}")
print(f"Volatilidade diária: {data['returns'].std():.4f}")
print(f"Retorno anualizado: {data['returns'].mean() * 252:.4f}")
print(f"Volatilidade anualizada: {data['returns'].std() * np.sqrt(252):.4f}")

## 3. Engenharia de Features

Agora vamos adicionar indicadores técnicos aos dados.

In [ ]:
from features.technical_indicators import TechnicalIndicators

# Adicionar indicadores técnicos
indicators = TechnicalIndicators()
data_with_features = indicators.add_all_features(data)

# Criar variável alvo
data_with_features['target'] = loader.create_target_variable(
    data_with_features,
    horizon=5,
    threshold=0.01
)

# Remover NaN
data_with_features = data_with_features.dropna()

print(f"Total de features: {len(data_with_features.columns)}")
print(f"Linhas após remoção de NaN: {len(data_with_features)}")

# Visualizar algumas features
data_with_features[['close', 'sma_20', 'sma_50', 'rsi_14', 'macd']].tail(10)

In [ ]:
# Visualizar indicadores técnicos
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Preço com médias móveis
axes[0].plot(data_with_features['date'], data_with_features['close'], label='Close', linewidth=1.5)
axes[0].plot(data_with_features['date'], data_with_features['sma_20'], label='SMA 20', linewidth=1.5)
axes[0].plot(data_with_features['date'], data_with_features['sma_50'], label='SMA 50', linewidth=1.5)
axes[0].set_title('Preço com Médias Móveis', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Preço ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(data_with_features['date'], data_with_features['rsi_14'], label='RSI 14', color='purple', linewidth=1.5)
axes[1].axhline(70, color='red', linestyle='--', label='Sobrecomprado')
axes[1].axhline(30, color='green', linestyle='--', label='Sobrevendido')
axes[1].set_title('Relative Strength Index (RSI)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('RSI')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MACD
axes[2].plot(data_with_features['date'], data_with_features['macd'], label='MACD', linewidth=1.5)
axes[2].plot(data_with_features['date'], data_with_features['macd_signal'], label='Signal', linewidth=1.5)
axes[2].bar(data_with_features['date'], data_with_features['macd_hist'], label='Histogram', alpha=0.3)
axes[2].set_title('MACD', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Data')
axes[2].set_ylabel('MACD')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Treinamento de Modelos

Vamos treinar múltiplos modelos de Machine Learning.

In [ ]:
from models.ml_models import TradingModel, EnsembleModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Preparar dados de treinamento
X_train, X_val, X_test, y_train, y_val, y_test = loader.prepare_training_data(
    data_with_features,
    target_col='target',
    test_size=0.2,
    validation_size=0.1
)

print(f"Training set: {len(X_train)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Test set: {len(X_test)} samples")

# Distribuição das classes
print("\nDistribuição do Target (treino):")
print(y_train.value_counts(normalize=True))

In [ ]:
# Treinar Random Forest
print("Treinando Random Forest...")
rf_model = TradingModel(model_type='random_forest')
rf_model.fit(X_train, y_train, X_val, y_val, n_estimators=100, max_depth=10)

# Treinar XGBoost
print("Treinando XGBoost...")
xgb_model = TradingModel(model_type='xgboost')
xgb_model.fit(X_train, y_train, X_val, y_val, n_estimators=100, max_depth=6)

# Treinar LightGBM
print("Treinando LightGBM...")
lgb_model = TradingModel(model_type='lightgbm')
lgb_model.fit(X_train, y_train, X_val, y_val, n_estimators=100, max_depth=6)

# Criar Ensemble
print("Criando Ensemble...")
ensemble = EnsembleModel([rf_model, xgb_model, lgb_model])
ensemble.fit(X_train, y_train, X_val, y_val)

print("\n✅ Todos os modelos treinados com sucesso!")

In [ ]:
# Avaliar modelos no conjunto de validação
print("📊 Performance dos Modelos (Validação):\n")
print("="*80)

models = [
    ('Random Forest', rf_model),
    ('XGBoost', xgb_model),
    ('LightGBM', lgb_model),
    ('Ensemble', ensemble)
]

results = []
for name, model in models:
    val_pred = model.predict(X_val)
    acc = accuracy_score(y_val, val_pred)
    prec = precision_score(y_val, val_pred, average='weighted', zero_division=0)
    rec = recall_score(y_val, val_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_val, val_pred, average='weighted', zero_division=0)
    
    results.append({
        'Model': name,
        'Accuracy': f"{acc:.3f}",
        'Precision': f"{prec:.3f}",
        'Recall': f"{rec:.3f}",
        'F1-Score': f"{f1:.3f}"
    })
    
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))
print("="*80)

In [ ]:
# Feature Importance
print("\n🔍 Top 15 Features Mais Importantes (Random Forest):\n")
top_features = rf_model.get_feature_importance(top_n=15)
print(top_features.to_string(index=False))

# Visualizar feature importance
plt.figure(figsize=(12, 6))
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importância')
plt.title('Top 15 Features Mais Importantes', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 5. Backtesting

Vamos testar a estratégia em dados históricos.

In [ ]:
from backtesting.backtest_engine import BacktestEngine

# Gerar previsões
test_predictions = ensemble.predict_proba(X_test)

# Criar sinais de trading
backtest = BacktestEngine(
    initial_capital=100000,
    commission=0.001,
    slippage=0.0005
)

signals = backtest.generate_signals_from_predictions(
    test_predictions,
    threshold=0.55
)

# Preparar dados de teste
test_data = data_with_features.iloc[-len(X_test):].copy()
test_data = test_data.reset_index(drop=True)
signals_series = pd.Series(signals).reset_index(drop=True)

# Executar backtest
results = backtest.run_backtest(test_data, signals_series, price_col='close')

print("✅ Backtest executado com sucesso!")

## 6. Análise de Resultados

Vamos analisar os resultados do backtest.

In [ ]:
# Exibir resultados
print("="*80)
print("📊 RESULTADOS DO BACKTEST")
print("="*80)
print(f"Capital Inicial:         ${backtest.initial_capital:,.2f}")
print(f"Valor Final da Carteira: ${results.equity_curve.iloc[-1]:,.2f}")
print(f"Retorno Total:           {results.total_return:.2%}")
print(f"Retorno Anualizado:      {results.annualized_return:.2%}")
print(f"Sharpe Ratio:            {results.sharpe_ratio:.2f}")
print(f"Drawdown Máximo:         {results.max_drawdown:.2%}")
print(f"Taxa de Acerto:          {results.win_rate:.2%}")
print(f"Profit Factor:           {results.profit_factor:.2f}")
print(f"Total de Trades:         {results.total_trades}")
print("="*80)

# Comparar com Buy & Hold
buy_hold_return = (test_data['close'].iloc[-1] / test_data['close'].iloc[0]) - 1
print(f"\nRetorno Buy & Hold:      {buy_hold_return:.2%}")
print(f"Superioridade da Estratégia: {(results.total_return - buy_hold_return):.2%}")

In [ ]:
# Visualizar curva de equity
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Equity curve
axes[0].plot(results.equity_curve.index, results.equity_curve.values, 
             label='Estratégia ML', linewidth=2, color='blue')

# Buy & Hold comparison
initial_price = test_data['close'].iloc[0]
buy_hold_equity = (test_data['close'] / initial_price) * backtest.initial_capital
buy_hold_equity = buy_hold_equity.reset_index(drop=True)
axes[0].plot(buy_hold_equity.index, buy_hold_equity.values, 
             label='Buy & Hold', linewidth=2, color='orange', linestyle='--')

axes[0].set_title('Curva de Equity', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Período')
axes[0].set_ylabel('Valor da Carteira ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].axhline(backtest.initial_capital, color='red', linestyle=':', linewidth=1, label='Capital Inicial')

# Drawdown
returns = results.equity_curve.pct_change()
cumulative = (1 + returns).cumprod()
running_max = cumulative.cummax()
drawdown = (cumulative - running_max) / running_max

axes[1].fill_between(drawdown.index, drawdown.values * 100, 0, 
                      alpha=0.3, color='red', label='Drawdown')
axes[1].plot(drawdown.index, drawdown.values * 100, color='darkred', linewidth=1.5)
axes[1].set_title('Drawdown', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Período')
axes[1].set_ylabel('Drawdown (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análise de trades
if len(results.trades) > 0:
    print("\n📝 Detalhes dos Trades:\n")
    print(results.trades.to_string(index=False))
    
    # Visualizar trades no gráfico
    plt.figure(figsize=(15, 6))
    plt.plot(test_data['close'].values, label='Preço', linewidth=1.5)
    
    # Marcar trades
    buy_trades = results.trades[results.trades['type'] == 'BUY']
    sell_trades = results.trades[results.trades['type'] == 'SELL']
    
    for _, trade in buy_trades.iterrows():
        if hasattr(trade['date'], 'value'):
            idx = test_data[test_data['date'] == trade['date']].index[0]
        else:
            idx = trade['date']
        plt.scatter(idx, trade['price'], color='green', marker='^', s=200, 
                   label='Buy' if _ == buy_trades.index[0] else '', zorder=5)
    
    for _, trade in sell_trades.iterrows():
        if hasattr(trade['date'], 'value'):
            idx = test_data[test_data['date'] == trade['date']].index[0]
        else:
            idx = trade['date']
        plt.scatter(idx, trade['price'], color='red', marker='v', s=200, 
                   label='Sell' if _ == sell_trades.index[0] else '', zorder=5)
    
    plt.title('Preço com Sinais de Compra/Venda', fontsize=14, fontweight='bold')
    plt.xlabel('Período')
    plt.ylabel('Preço ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("\n⚠️ Nenhum trade foi executado no período de teste.")

## 📊 Conclusões

Este notebook demonstrou o pipeline completo de uma estratégia de trading com Machine Learning:

1. ✅ Carregamento de dados históricos
2. ✅ Análise exploratória de dados
3. ✅ Engenharia de features com indicadores técnicos
4. ✅ Treinamento de múltiplos modelos de ML
5. ✅ Backtesting da estratégia
6. ✅ Análise detalhada dos resultados

### Próximos Passos:

- Experimente com diferentes tickers e períodos
- Ajuste os hiperparâmetros dos modelos
- Teste diferentes thresholds para geração de sinais
- Adicione mais indicadores técnicos
- Implemente estratégias de gestão de risco

---

**⚠️ Aviso Legal:** Este notebook é apenas para fins educacionais. Não constitui conselho financeiro. Sempre faça sua própria pesquisa antes de investir.